In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import stats
pd.set_option("display.max_columns", 1000)
import math
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import classification_report

countries = pd.read_csv("countries.csv")
test = pd.read_csv("test_users.csv")
train = pd.read_csv("train_users_2.csv")
sessions = pd.read_csv("sessions.csv")
test.head()
train.head()
sessions.head()
print(test.shape)
print(train.shape)
print(sessions.shape)
sessions.isnull().sum()
sessions = sessions[sessions.user_id.notnull()]
sessions.isnull().sum()
sessions[sessions.action.isnull()].action_type.value_counts()
sessions.loc[sessions.action.isnull(), 'action'] = 'message'
sessions.isnull().sum()
print(sessions[sessions.action_type.isnull()].action.value_counts())
print()
print(sessions[sessions.action_detail.isnull()].action.value_counts())
def each_user_common_value(dataframe, feature): 
    new_df = pd.DataFrame(dataframe.groupby(['user_id','action'])[feature].value_counts())
    new_df['index_tuple'] = new_df.index 
    new_df['count'] = new_df[feature]
    new_columns = ['user_id','action',feature]
    for n,col in enumerate(new_columns):
        new_df[col] = new_df.index_tuple.apply(lambda index_tuple: index_tuple[n])
    new_df = new_df.reset_index(drop = True)
    new_df = new_df.drop(['index_tuple'], axis = 1)     
    new_df_max = pd.DataFrame(new_df.groupby(['user_id','action'], as_index = False)['count'].max())
    new_df_max = new_df_max.merge(new_df, on = ['user_id','action','count'])
    new_df_max = new_df_max.drop('count', axis = 1)   
    dataframe = dataframe.merge(new_df_max, left_on = ['user_id','action'], right_on = ['user_id','action'], how = 'left')
    return dataframe
sessions = each_user_common_value(sessions, 'action_type')
print("action_type is complete.")

sessions = each_user_common_value(sessions, 'action_detail')
print("action_detail is complete.")
sessions.loc[sessions.action_type_x.isnull(), 'action_type_x'] = sessions.action_type_y
sessions.loc[sessions.action_detail_x.isnull(), 'action_detail_x'] = sessions.action_detail_y

sessions['action_type'] = sessions.action_type_x
sessions['action_detail'] = sessions.action_detail_x
sessions = sessions.drop(['action_type_x','action_type_y','action_detail_x','action_detail_y'], axis = 1)
sessions.isnull().sum()
def all_users_common_value(dataframe, feature):
    new_df = pd.DataFrame(dataframe.groupby('action')[feature].value_counts())
    
    new_df['index_tuple'] = new_df.index 
    new_df['count'] = new_df[feature]
    
    new_columns = ['action',feature]
    for n,col in enumerate(new_columns):
        new_df[col] = new_df.index_tuple.apply(lambda index_tuple: index_tuple[n])
    
    new_df = new_df.reset_index(drop = True)
    new_df = new_df.drop(['index_tuple'], axis = 1) 
    
    new_df_max = pd.DataFrame(new_df.groupby('action', as_index = False)['count'].max())
    new_df_max = new_df_max.merge(new_df, on = ['action','count'])
    new_df_max = new_df_max.drop('count', axis = 1)
    
    dataframe = dataframe.merge(new_df_max, left_on = 'action', right_on = 'action', how = 'left')
    
    return dataframe
sessions = all_users_common_value(sessions, 'action_type')
print("action_type is complete.")
sessions = all_users_common_value(sessions, 'action_detail')
print("action_detail is complete.")
sessions.loc[sessions.action_type_x.isnull(), 'action_type_x'] = sessions.action_type_y
sessions.loc[sessions.action_detail_x.isnull(), 'action_detail_x'] = sessions.action_detail_y

sessions['action_type'] = sessions.action_type_x
sessions['action_detail'] = sessions.action_detail_x
sessions = sessions.drop(['action_type_x','action_type_y','action_detail_x','action_detail_y'], axis = 1)
sessions.isnull().sum()
sessions[sessions.action_type.isnull()].action.value_counts()
sessions.action.value_counts()
print(sessions[sessions.action == 'similar_listings'].action_type.value_counts())
print(sessions[sessions.action == 'similar_listings'].action_detail.value_counts())
sessions.loc[sessions.action == 'similar_listings_v2', 'action_type'] = "data"
sessions.loc[sessions.action == 'similar_listings_v2', 'action_detail'] = "similar_listings"

sessions.loc[sessions.action == 'lookup', 'action_type'] = "lookup"
sessions.loc[sessions.action == 'lookup', 'action_detail'] = "lookup"

sessions.loc[sessions.action == 'track_page_view', 'action_type'] = "track_page_view"
sessions.loc[sessions.action == 'track_page_view', 'action_detail'] = "track_page_view"

sessions.action_type = sessions.action_type.fillna("missing")
sessions.action_detail = sessions.action_detail.fillna("missing")
sessions.isnull().sum()
median_duration = pd.DataFrame(sessions.groupby('action', as_index = False)['secs_elapsed'].median())
median_duration.head()
sessions = sessions.merge(median_duration, left_on = 'action', right_on = 'action', how = 'left')
print("Merge complete.")
sessions.loc[sessions.secs_elapsed_x.isnull(), 'secs_elapsed_x'] = sessions.secs_elapsed_y
print("Nulls are filled.")
sessions['secs_elapsed'] = sessions.secs_elapsed_x
print("Column is created.")
sessions = sessions.drop(['secs_elapsed_x','secs_elapsed_y'], axis = 1)
print("Columns are dropped.")
sessions.isnull().sum()
sessions.head()
sessions_summ = pd.DataFrame(sessions.user_id.value_counts(sort = False))
sessions_summ['action_count'] = sessions_summ.user_id
sessions_summ['user_id'] = sessions_summ.index
sessions_summ = sessions_summ.reset_index(drop = True)
sessions_summ.head()
user_active = pd.DataFrame(sessions.groupby('user_id').secs_elapsed.sum())
user_active['user_id'] = user_active.index
user_active = user_active.reset_index(drop = True)
sessions_summ = sessions_summ.merge(user_active)
sessions_summ['duration'] = sessions_summ.secs_elapsed
sessions_summ = sessions_summ.drop("secs_elapsed", axis = 1)
sessions_summ.head()
def most_frequent_value(dataframe, feature):
    new_df = pd.DataFrame(sessions.groupby('user_id')[feature].value_counts())
    new_df['index_tuple'] = new_df.index
    new_columns = ['user_id',feature]
    for n,col in enumerate(new_columns):
        new_df[col] = new_df.index_tuple.apply(lambda index_tuple: index_tuple[n])
    
    new_df = new_df.reset_index(drop = True)
    new_df = new_df.drop('index_tuple', axis = 1)
    new_df = new_df.groupby('user_id').first()
    
    new_df['user_id'] = new_df.index
    new_df = new_df.reset_index(drop = True)
    
    dataframe = dataframe.merge(new_df)
    
    return dataframe
sessions_f = ['action','action_type','action_detail','device_type']

for f in sessions_f:
    sessions_summ = most_frequent_value(sessions_summ, f)
    print("{} is complete.".format(f))
sessions_summ.head()
sessions_summ.head()
def unique_features(feature, feature_name, dataframe):
    unique_feature = pd.DataFrame(sessions.groupby('user_id')[feature].unique())
    unique_feature['user_id'] = unique_feature.index
    unique_feature = unique_feature.reset_index(drop = True)
    unique_feature[feature_name] = unique_feature[feature].map(lambda x: len(x))
    unique_feature = unique_feature.drop(feature, axis = 1)
    dataframe = dataframe.merge(unique_feature, on = 'user_id')
    return dataframe
sessions_summ = unique_features('action', 'unique_actions', sessions_summ)
print("action is complete.")
sessions_summ = unique_features('action_type', 'unique_action_types', sessions_summ)
print("action_type is complete.")
sessions_summ = unique_features('action_detail', 'unique_action_details', sessions_summ)
print("action_detail is complete.")
sessions_summ = unique_features('device_type', 'unique_device_types', sessions_summ)
print("device_type is complete.")
sessions_summ.head()
max_durations = pd.DataFrame(sessions.groupby(['user_id'], as_index = False)['secs_elapsed'].max())
sessions_summ = sessions_summ.merge(max_durations, on = 'user_id')
sessions_summ['max_time'] = sessions_summ.secs_elapsed
sessions_summ = sessions_summ.drop('secs_elapsed', axis = 1)

print("max_durations is complete.")

min_durations = pd.DataFrame(sessions.groupby(['user_id'], as_index = False)['secs_elapsed'].min())
sessions_summ = sessions_summ.merge(min_durations, on = 'user_id')
sessions_summ['min_time'] = sessions_summ.secs_elapsed
sessions_summ = sessions_summ.drop('secs_elapsed', axis = 1)

print("min_durations is complete.")
sessions_summ['avg_duration'] = sessions_summ.duration / sessions_summ.action_count
sessions_summ.head(5)
apple_device = ['Mac Desktop','iPhone','iPdad Tablet','iPodtouch']
desktop_device = ['Mac Desktop','Windows Desktop','Chromebook','Linux Desktop']
tablet_device = ['Android App Unknown Phone/Tablet','iPad Tablet','Tablet']
mobile_device = ['Android Phone','iPhone','Windows Phone','Blackberry','Opera Phone']

device_types = {'apple_device': apple_device, 
                'desktop_device': desktop_device,
                'tablet_device': tablet_device,
                'mobile_device': mobile_device}

for device in device_types:
    sessions_summ[device] = 0
    sessions_summ.loc[sessions_summ.device_type.isin(device_types[device]), device] = 1
sessions_summ.head()
sessions_summ.isnull().sum()
train1 = pd.merge(train,sessions_summ, left_on = train['id'], right_on = sessions_summ['user_id'], how = 'inner')
train2 = train[~train.id.isin(train1.id)]
train = pd.concat([train1, train2])

test1 = test.merge(sessions_summ, left_on = test['id'], right_on = sessions_summ['user_id'], how = 'inner')
test2 = test[~test.id.isin(test1.id)]
test = pd.concat([test1, test2])
df = pd.concat([train,test], sort =False)
df.head()
df.shape
df.isnull().sum()
df = df.drop('user_id', axis = 1)
df = df.drop('key_0', axis = 1)
def missing_session_data_categorical(feature):
    return df[feature].fillna("missing")

def missing_session_data_continuous(feature):
    return df[feature].fillna(0)
session_features_cat = ['action','action_detail','action_type','device_type']
session_features_cont = ['action_count','apple_device','desktop_device','mobile_device','tablet_device',
                         'duration','avg_duration','max_time','min_time','unique_action_details',
                         'unique_action_types','unique_actions','unique_device_types']

for feature in session_features_cat:
    df[feature] = missing_session_data_categorical(feature)
    
for feature in session_features_cont:
    df[feature] = missing_session_data_continuous(feature)
df.isnull().sum()
df.action_count.describe()
df[df.action_count > 0].action_count.describe()
df['action_count_quartile'] = df.action_count.map(lambda x: 0 if x == 0 else (
                                                            1 if x <= 17 else (
                                                            2 if x <= 43 else (
                                                            3 if x <= 97 else 4))))
df[df.age.notnull()].age.describe()
df.loc[df.age > 80, 'age'] = 80
df[df.age.notnull()].age.describe()
for feature in df.columns:
    if(df[feature].dtype == float or df[feature].dtype == int):
        correlation = stats.pearsonr(df[df.age.notnull()].age, df[df.age.notnull()][feature])
        print("Correlation with {} = {}".format(feature, correlation))
df['age_group'] = df.age.map(lambda x: 0 if math.isnan(x) else (
                                       1 if x < 18 else (
                                       2 if x <= 33 else (
                                       3 if x <= 42 else 4))))
df.age = df.age.fillna(33)
df.age.isnull().sum()
print(df.duration.describe())
print()
print(df.avg_duration.describe())
print(np.percentile(df.duration, 50))
print(np.percentile(df.duration, 51))
print(np.percentile(df.duration, 75))
print()
print(np.percentile(df.avg_duration, 50))
print(np.percentile(df.avg_duration, 51))
print(np.percentile(df.avg_duration, 75))
df['duration_group'] = df.duration.map(lambda x: 0 if x == 0 else (
                                                 1 if x <= 877166.25 else 2))

df['avg_duration_group'] = df.avg_duration.map(lambda x: 0 if x == 0 else (
                                                         1 if x <= 16553.7889474 else 2))
print(df.duration_group.value_counts())
print()
print(df.avg_duration_group.value_counts())
df.gender.value_counts()
df.mobile_device.value_counts()
df.signup_flow.value_counts()
df['signup_flow_simple'] = df.signup_flow.map(lambda x: 0 if x == 0 else 1)
df['signup_flow_simple'].value_counts()
df.tablet_device.value_counts()
df.date_account_created = pd.to_datetime(df.date_account_created, format='%Y-%m-%d')
df.date_first_booking = pd.to_datetime(df.date_first_booking, format='%Y-%m-%d')
print(df.date_account_created.min())
print(df.date_account_created.max())
print()
print(df.date_first_booking.min())
print(df.date_first_booking.max())
calendar = USFederalHolidayCalendar()
holidays = calendar.holidays(start = df.date_account_created.min(), 
                             end = df.date_first_booking.max())

us_bd = CustomBusinessDay(calendar = USFederalHolidayCalendar())
business_days = pd.DatetimeIndex(start = df.date_account_created.min(), 
                                 end = df.date_first_booking.max(), 
                                 freq = us_bd)
df['year_account_created'] = df.date_account_created.dt.year
df['month_account_created'] = df.date_account_created.dt.month
df['weekday_account_created'] = df.date_account_created.dt.weekday
df['business_day_account_created'] = df.date_account_created.isin(business_days)
df['business_day_account_created'] = df.business_day_account_created.map(lambda x: 1 if x == True else 0)
df['holiday_account_created'] = df.date_account_created.isin(holidays)
df['holiday_account_created'] = df.holiday_account_created.map(lambda x: 1 if x == True else 0)

df['year_first_booking'] = df.date_first_booking.dt.year
df['month_first_booking'] = df.date_first_booking.dt.month
df['weekday_first_booking'] = df.date_first_booking.dt.weekday
df['business_day_first_booking'] = df.date_first_booking.isin(business_days)
df['business_day_first_booking'] = df.business_day_first_booking.map(lambda x: 1 if x == True else 0)
df['holiday_first_booking'] = df.date_first_booking.isin(holidays)
df['holiday_first_booking'] = df.holiday_first_booking.map(lambda x: 1 if x == True else 0)

df = df.drop(["date_first_booking","date_account_created"], axis = 1)
df.isnull().sum()
df.year_first_booking = df.year_first_booking.fillna(min(df.year_first_booking) - 1)
df.month_first_booking = df.month_first_booking.fillna(min(df.month_first_booking) - 1)
df.weekday_first_booking += 1
df.weekday_first_booking = df.weekday_first_booking.fillna(0)
df.isnull().sum()
df.first_affiliate_tracked.value_counts()
df.first_affiliate_tracked = df.first_affiliate_tracked.fillna("untracked")
df.isnull().sum()
df.head()
df.first_browser.value_counts()
mobile_browsers = ['Mobile Safari','Chrome Mobile','IE Mobile','Mobile Firefox','Android Browser']
df.loc[df.first_browser.isin(mobile_browsers), "first_browser"] = "Mobile"
cut_off = 1378

other_browsers = []
for browser, count in df.first_browser.value_counts().iteritems():
    if count < cut_off:
        other_browsers.append(browser)
df.loc[df.first_browser.isin(other_browsers), "first_browser"] = "Other"
df.first_browser.value_counts()
df.language.value_counts()
other_languages = []
for language, count in df.language.value_counts().iteritems():
    if count < 275:
        other_languages.append(language)
    
print(other_languages)

df.loc[df.language.isin(other_languages), "language"] = "Other"
df.language.value_counts()
df['not_English'] = df.language.map(lambda x: 0 if x == 'en' else 1)
df.action.value_counts()
other_actions = []
for action, count in df.action.value_counts().iteritems():
    if count < cut_off:
        other_actions.append(action)
    
print(other_actions)

df.loc[df.action.isin(other_actions), "action"] = "Other"
df.action.value_counts()
df.action_detail.value_counts()
other_action_details = []
for action_detail, count in df.action_detail.value_counts().iteritems():
    if count < cut_off:
        other_action_details.append(action_detail)
    
print(other_action_details)

df.loc[df.action_detail.isin(other_action_details), "action_detail"] = "Other"
df.action_detail.value_counts()
df.action_type.value_counts()
other_action_types = []
for action_type, count in df.action_type.value_counts().iteritems():
    if count < 1378:
        other_action_types.append(action_type)
    
print(other_action_types)

df.loc[df.action_type.isin(other_action_types), "action_type"] = "Other"
df.action_type.value_counts()
df.affiliate_provider.value_counts()
other_affiliate_providers = []
for affiliate_provider, count in df.affiliate_provider.value_counts().iteritems():
    if count < cut_off:
        other_affiliate_providers.append(affiliate_provider)
    
print(other_affiliate_providers)

df.loc[df.affiliate_provider.isin(other_affiliate_providers), "affiliate_provider"] = "other"
df.affiliate_provider.value_counts()
df.device_type.value_counts()
other_device_types = []
for device_type, count in df.device_type.value_counts().iteritems():
    if count < 1378:
        other_device_types.append(device_type)
    
print(other_device_types)

df.loc[df.device_type.isin(other_device_types), "device_type"] = "Other"
df.device_type.value_counts()
df.signup_method.value_counts()
labels = pd.DataFrame(df.country_destination)
df = df.drop("country_destination", axis = 1)
labels.head()
df = df.drop('id', axis = 1)
df.dtypes
cont_features = []
cat_features = []

for feature in df.columns:
    print(feature)
    if df[feature].dtype == float or df[feature].dtype == int:
        cont_features.append(feature)
    elif df[feature].dtype == object:
        cat_features.append(feature)
print(cat_features)
print()
print(cont_features)
print()
print(len(cat_features) + len(cont_features))
print(df.shape[1])
date_features = ['year_account_created','month_account_created','weekday_account_created',
                      'year_first_booking','month_first_booking','weekday_first_booking']
for feature in date_features:
    if feature in cont_features:
        cont_features.remove(feature)
    cat_features.append(feature)
for feature in cat_features:
    dummies = pd.get_dummies(df[feature], prefix = feature, sparse=False, drop_first = False, dtype='float')
    df = df.drop(feature, axis = 1)
    df = pd.concat([df, dummies], axis=1)
    print("{} is complete".format(feature))
min_max_scaler = preprocessing.MinMaxScaler()
features_remain =['action_count', 'age', 'apple_device', 'avg_duration', 'desktop_device', 'duration', 'max_time',
                  'min_time', 'mobile_device', 'signup_flow', 'tablet_device', 'timestamp_first_active',
                  'unique_action_details', 'unique_action_types', 'unique_actions', 'unique_device_types',
                  'action_count_quartile', 'age_group', 'duration_group', 'avg_duration_group', 'signup_flow_simple',
                  'business_day_account_created', 'holiday_account_created','business_day_first_booking', 
                  'holiday_first_booking', 'not_English']
for rf in features_remain:
    df.loc[:,rf] = min_max_scaler.fit_transform(df[[rf]])
df.dtypes
df.dtypes
df_train = df[:len(train)]
df_test = df[len(train):]
print(len(train))
y = labels[:len(train)]
y_dummies = pd.get_dummies(y, drop_first = False, dtype=float)
y = pd.concat([y, y_dummies], axis=1)
y = y.drop("country_destination", axis = 1)
y.head()
print(df_train.shape)
print(df_test.shape)
print(y.shape)
train.country_destination.value_counts()
y[y.columns[0]] *= len(y)/539
y[y.columns[1]] *= len(y)/1428
y[y.columns[2]] *= len(y)/1061
y[y.columns[3]] *= len(y)/2249
y[y.columns[4]] *= len(y)/5023
y[y.columns[5]] *= len(y)/2324
y[y.columns[6]] *= len(y)/2835
y[y.columns[7]] *= len(y)/124543
y[y.columns[8]] *= len(y)/762
y[y.columns[9]] *= len(y)/217
y[y.columns[10]] *= len(y)/62376
y[y.columns[11]] *= len(y)/10094
totals = []
for i in range(12):
    totals.append(sum(y[y.columns[i]]))
totals
df_train.head()
x_train, x_test, y_train, y_test = train_test_split(df_train, y, test_size = 0.2, random_state = 2)
x_train.head()
inputX = x_train.as_matrix()
inputY = y_train.as_matrix()
inputX_test = x_test.as_matrix()
inputY_test = y_test.as_matrix()
input_nodes = 186

mulitplier = 1.33

hidden_nodes1 = 50
hidden_nodes2 = round(hidden_nodes1 * mulitplier)

pkeep = tf.placeholder(tf.float32)
std = 1

features = tf.placeholder(tf.float32, [None, input_nodes])

W1 = tf.Variable(tf.truncated_normal([input_nodes, hidden_nodes1], stddev = std))
b1 = tf.Variable(tf.zeros([hidden_nodes1]))
y1 = tf.nn.sigmoid(tf.matmul(features, W1) + b1)

W2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev = std))
b2 = tf.Variable(tf.zeros([hidden_nodes2]))
y2 = tf.nn.sigmoid(tf.matmul(y1, W2) + b2)

W3 = tf.Variable(tf.truncated_normal([hidden_nodes2, 12], stddev = std)) 
b3 = tf.Variable(tf.zeros([12]))
y3 = tf.nn.softmax(tf.matmul(y2, W3) + b3)

predictions = y3
labels = tf.placeholder(tf.float32, [None, 12])
training_epochs = 2000
training_dropout = 0.6 
display_step = 10
n_samples = inputY.shape[1]
batch = tf.Variable(0)

learning_rate = tf.train.exponential_decay(
  0.05,              #Base learning rate.
  batch,             #Current index into the dataset.
  len(inputX),       #Decay step.
  0.95,              #Decay rate.
  staircase=False)
correct_prediction = tf.equal(tf.argmax(predictions,1), tf.argmax(labels,1))
correct_top5 = tf.nn.in_top_k(predictions, tf.argmax(labels, 1), k = 5)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_top5 = tf.reduce_mean(tf.cast(correct_top5, tf.float32))

print('Accuracy function created.')

cross_entropy = -tf.reduce_sum(labels * tf.log(tf.clip_by_value(predictions,1e-10,1.0)))

loss = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

accuracy_summary = [] 
accuracy_top5_summary = [] 
loss_summary = []

test_accuracy_summary = []
test_accuracy_top5_summary = [] 
test_loss_summary = [] 

init = tf.global_variables_initializer()

for i in range(training_epochs):  
    session.run([optimizer], 
                feed_dict={features: inputX, 
                           labels: inputY,
                           pkeep: training_dropout})

    if (i) % display_step == 0:
        train_accuracy, train_accuracy_top5, newLoss = session.run([accuracy,accuracy_top5,loss], 
                                                                   feed_dict={features: inputX, 
                                                                              labels: inputY,
                                                                              pkeep: training_dropout})
        print ("Epoch:", i,
               "Accuracy =", "{:.6f}".format(train_accuracy), 
               "Top 5 Accuracy =", "{:.6f}".format(train_accuracy_top5),
               "Loss = ", "{:.6f}".format(newLoss))
        accuracy_summary.append(train_accuracy)
        accuracy_top5_summary.append(train_accuracy_top5)
        loss_summary.append(newLoss)
        
        test_accuracy,test_accuracy_top5,test_newLoss = session.run([accuracy,accuracy_top5,loss], 
                                                              feed_dict={features: inputX_test, 
                                                                         labels: inputY_test,
                                                                         pkeep: 1})
        print ("Epoch:", i,
               "Test-Accuracy =", "{:.6f}".format(test_accuracy), 
               "Test-Top 5 Accuracy =", "{:.6f}".format(test_accuracy_top5),
               "Test-Loss = ", "{:.6f}".format(test_newLoss))
        test_accuracy_summary.append(test_accuracy)
        test_accuracy_top5_summary.append(test_accuracy_top5)
        test_loss_summary.append(test_newLoss)

print()
print ("Optimization Finished!")
training_accuracy, training_top5_accuracy = session.run([accuracy,accuracy_top5], 
                                feed_dict={features: inputX, labels: inputY, pkeep: training_dropout})
print ("Training Accuracy=", training_accuracy)
print ("Training Top 5 Accuracy=", training_top5_accuracy)
print()
testing_accuracy, testing_top5_accuracy = session.run([accuracy,accuracy_top5], 
                                                       feed_dict={features: inputX_test, 
                                                                  labels: inputY_test,
                                                                  pkeep: 1})
print ("Testing Accuracy=", testing_accuracy)
print ("Testing Top 5 Accuracy=", testing_top5_accuracy)
testing_predictions, testing_labels = session.run([tf.argmax(predictions,1), tf.argmax(labels,1)], 
                                                  feed_dict={features: inputX_test,
                                                             labels: inputY_test,
                                                             pkeep: 1})

print(classification_report(testing_labels, testing_predictions, target_names=y.columns))
f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(10,8))

ax1.plot(accuracy_summary)
ax1.plot(test_accuracy_summary)
ax1.set_title('Top 1 Accuracy')

ax2.plot(accuracy_top5_summary)
ax2.plot(test_accuracy_top5_summary)
ax2.set_title('Top 5 Accuracy')

ax3.plot(loss_summary)
ax3.plot(test_loss_summary)
ax3.set_title('Loss')

plt.xlabel('Epochs (x10)')
plt.show()
test_final = df_test.as_matrix()
final_probabilities = session.run(predictions, feed_dict={features: test_final,
                                                          pkeep: 1})
final_probabilities[0]
le = LabelEncoder()
fit_labels = le.fit_transform(train.country_destination) 

test_getIDs = pd.read_csv("test_users.csv")
testIDs = test_getIDs['id']

ids = []  
countries = []  
for i in range(len(testIDs)):
    idx = testIDs[i]
    ids += [idx] * 5
    countries += le.inverse_transform(np.argsort(final_probabilities[i])[::-1])[:5].tolist()
    if i % 10000 == 0:
        print ("Percent complete: {}%".format(round(i / len(test),4)*100))

submission = pd.DataFrame(np.column_stack((ids, countries)), columns=['id', 'country'])
submission.to_csv('submission.csv',index=False)
submission.head(25)
submission.country.value_counts()
train.country_destination.value_counts()

